# Setup

Please ensure you have imported a Gemini API key from AI Studio.
You can do this directly in the Secrets tab on the left.

After doing so, please run the setup cell below.

In [9]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

import os
from google.colab import userdata
from google.colab import drive
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")

drive.mount("/content/drive")
# Please ensure that uploaded files are available in the AI Studio folder or change the working folder.
os.chdir("/content/drive/MyDrive/Google AI Studio")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Generated Code

In [10]:
import re
import pandas as pd
from tqdm.notebook import tqdm # Import tqdm
from google import genai
from google.genai import types
import os


# Daftar pola yang menandakan artikel perlu dibersihkan oleh LLM.
# re.IGNORECASE digunakan agar tidak peduli huruf besar/kecil.
DIRTY_PATTERNS = [
    re.compile(r"KOMPAS\.com|TRIBUNNEWS\.COM|Liputan6\.com|detikcom", re.IGNORECASE),
    re.compile(r"Advertisement|Scroll untuk melanjutkan|Download aplikasi", re.IGNORECASE),
    re.compile(r"Baca Juga:|Simak Juga:|Lihat Juga:", re.IGNORECASE),
    re.compile(r"\[Gambas:|\[FOTO\]|\[VIDEO\]", re.IGNORECASE),
    re.compile(r"Penulis:|Editor:|Sumber:", re.IGNORECASE),
    re.compile(r"Beri komentar|Bagikan artikel ini", re.IGNORECASE),
    re.compile(r"Disclaimer", re.IGNORECASE),
]

def perlu_dibersihkan_llm(teks_artikel: str) -> bool:
    """
    Fungsi untuk memeriksa apakah sebuah artikel mengandung pola "kotor".
    Mengembalikan True jika ditemukan, False jika tidak.
    """
    if pd.isna(teks_artikel):
        return False  # Treat NaN as clean (or handle as needed)

    for pattern in DIRTY_PATTERNS:
        if pattern.search(teks_artikel):
            # Jika satu saja pola ditemukan, langsung kembalikan True.
            # Tidak perlu cek sisanya.
            return True
    return False

def perbaikan_sederhana(teks_artikel: str) -> str:
    """
    Fungsi untuk perbaikan kecil yang tidak memerlukan LLM,
    misalnya mengganti tanda petik.
    """
    if pd.isna(teks_artikel):
        return teks_artikel # Return NaN if input is NaN

    teks_bersih = teks_artikel.replace("“", '"').replace("”", '"')
    teks_bersih = teks_bersih.replace("‘", "'").replace("’", "'")
    return teks_bersih

def generate(isi_berita):
    """
    Fungsi untuk memanggil LLM untuk membersihkan teks berita.
    """
    if pd.isna(isi_berita):
        return isi_berita # Return NaN if input is NaN

    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.5-flash-lite-preview-06-17"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=f"""Berikut teks yang perlu dibersihkan:
\"\"\"
{isi_berita}
\"\"\"
"""),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0,
        thinking_config = types.ThinkingConfig(
            thinking_budget=0,
        ),
        response_mime_type="text/plain",
        system_instruction=[
            types.Part.from_text(text="""
            Anda adalah AI penyunting teks berita. Tugas Anda adalah mengubah artikel mentah menjadi teks berita yang bersih dan profesional. Ikuti semua aturan di bawah ini secara ketat.

            **Urutan Penyuntingan:**

            **A. Bagian Awal & Judul Teks**
            1.  **HAPUS DATELINE:** Hapus baris pembuka yang berisi nama media dan lokasi (misal: "KOMPAS.com, Jakarta - ", "TRIBUNNEWS.COM - ").

            **B. Bagian Isi Teks**
            2.  **HAPUS IKLAN & PROMOSI:** Hapus semua bentuk iklan (misal: "Advertisement", "Scroll untuk melanjutkan") dan promosi platform lain (misal: "Download aplikasi...", "Saksikan di YouTube...").
            3.  **HAPUS REFERENSI INTERNAL:** Hapus frasa rujukan artikel lain (misal: "Baca Juga:", "Simak Juga:", dan variasinya) beserta kalimat atau tautan yang mengikutinya.
            4.  **HAPUS TAG MEDIA:** Hapus semua placeholder media (misal: "[Gambas:Video 20detik]", "[FOTO]").
            5.  **PERBAIKI TEKS:**
                * **Kesalahan Tik (Typo):** Koreksi ejaan kata yang salah (misal: `pmerintah` → `pemerintah`). Gunakan konteks untuk akurasi.
                * **Spasi:** Perbaiki spasi yang hilang (misal: `LIVEmalam` → `LIVE malam`) dan hapus spasi ganda.
                * **Singkatan:** JANGAN ubah singkatan umum (misal: "yg", "utk", "dll", "pemilu").

            **C. Bagian Akhir Teks**
            6.  **HAPUS AJAKAN INTERAKTIF:** Hapus kalimat ajakan (misal: "Beri komentar di bawah.", "Bagikan artikel ini.").
            7.  **HAPUS DISCLAIMER:** Hapus seluruh kalimat atau paragraf disclaimer.
            8.  **HAPUS KREDIT:** Hapus baris kredit di akhir teks (misal: "Penulis:", "Editor:", "Sumber:").

            **D. Aturan Finalisasi & Output**
            9.  **STANDARKAN FORMAT:** Ganti semua tanda petik (`“...“`, `‘...’`) menjadi tanda petik lurus (`"..."`).
            10. **PERTAHANKAN INTI BERITA:** Jaga keutuhan semua fakta, data, dan kutipan narasumber.

            **PERINTAH OUTPUT FINAL:**
            Kirimkan **HANYA** teks berita hasil akhir yang sudah bersih. Jangan menyertakan komentar, penjelasan, atau format tambahan apa pun.
            """),
        ],
    )

    cleaned_text = ""
    try:
        for chunk in client.models.generate_content_stream(
            model=model,
            contents=contents,
            config=generate_content_config,
        ):
            if chunk.text is not None:
              cleaned_text += chunk.text
        return cleaned_text
    except Exception as e:
        print(f"Error processing text: {e}")
        return isi_berita # Return original text in case of error


if __name__ == "__main__":
    # Load your dataframe here. Replace 'your_dataset.csv' with your actual file path.
    # For example:
    df = pd.read_csv('/content/drive/MyDrive/KlikBERT/final_dataset_part1.csv')

    # Create a new column for cleaned text
    df['isi_berita_bersih'] = None

    # Apply the cleaning logic row by row and assign the result back to the DataFrame
    tqdm.pandas() # Initialize pandas integration with tqdm

    def apply_cleaning_and_assign(row):
        if perlu_dibersihkan_llm(row['isi']):
            # If the article needs LLM cleaning, call the generate function
            cleaned_text = generate(row['isi'])
        else:
            # Otherwise, just apply simple fixes
            cleaned_text = perbaikan_sederhana(row['isi'])
        # Assign the cleaned text back to the row
        row['isi_berita_bersih'] = cleaned_text
        return row # Return the modified row

    # Use apply to update the DataFrame in place
    df = df.progress_apply(apply_cleaning_and_assign, axis=1)


    # Display the updated DataFrame
    display(df)

    # Save the cleaned data to a new CSV file
    df.to_csv('/content/drive/MyDrive/Google AI Studio/final_dataset_part1_cleaned.csv', index=False)

  0%|          | 0/22789 [00:00<?, ?it/s]

,Unnamed: 0,judul,isi,url,isi_berita_bersih
0,0,Bongkahan Semen dari Atap Masjid Agung Abdya J...,Laporan Masrian Mizani I Aceh Barat Daya\nSERA...,https://aceh.tribunnews.com/2025/06/09/bongkah...,Bongkahan semen dari atap Masjid Agung Baitul ...
1,1,Cerita Afgan Berangkat Haji Sendiri: Modal Nek...,"""Alhamdulillah completed my first Hajj. It’s b...",https://aceh.tribunnews.com/2025/06/09/cerita-...,Penyanyi Afgan merekam video saat dirinya berj...
2,2,"Cuaca Buruk, Dua Keberangkatan Kapal Cepat dar...",Kepala UPTD Penyelenggaraan Pelabuhan Penyeber...,https://aceh.tribunnews.com/2025/06/09/cuaca-b...,Dua trip kapal cepat dari Pelabuhan Penyeberan...
3,3,Diprotes Maxime Bottier Gegara Tak Pakai Cinci...,"""Sudah kawin masak nggak dipakai cincinnya,"" k...",https://aceh.tribunnews.com/2025/06/09/diprote...,Pasangan artis Luna Maya dan Maxime Bouttier b...
4,4,"Gajah Kembali Obrak-abrik Kebun, Kali Ini 3 He...",Ia menyebutkan bahwa kebun sawit milik sejumla...,https://aceh.tribunnews.com/2025/06/09/gajah-k...,Kawanan gajah Sumatera kembali merusak kebun m...
...,...,...,...,...,...
22784,22784,Perusahaan Raksasa di Balik Keluhan Amerika So...,Tak perlu pidato panjang lebar dan berapi-api ...,https://newsletter.tempo.co/read/2006642/perus...,Tak perlu pidato panjang lebar dan berapi-api ...
22785,22785,Burung Air Mentok Rimba Terancam Punah dari Ha...,BARANGKALI banyak dari kita yang baru mendenga...,https://newsletter.tempo.co/read/2006721/burun...,BARANGKALI banyak dari kita yang baru mendenga...
22786,22786,"Bom Waktu Hubungan Prabowo-Gibran, Relasi Kedu...","BEBERAPA waktu lalu, timTempomulai mendengar k...",https://newsletter.tempo.co/read/2006727/bom-w...,"BEBERAPA waktu lalu, timTempomulai mendengar k..."
22787,22787,Tangan Terampil di Balik Karya Seni Sang Senim...,APAKAH kau tahu bahwa sejumlah karya seni yang...,https://newsletter.tempo.co/read/2006733/tanga...,APAKAH kau tahu bahwa sejumlah karya seni yang...


In [3]:
# cek NaN
df.isnull().sum()

,0
judul,0
isi,0
url,0
isi_berita_bersih,20
